In [13]:
import logging
import os
from datetime import datetime  # timezone

import boto3
import botocore
from botocore.exceptions import ClientError

# import AssetFunctions

region_name = 'ap-south-1'
aws_access_key_id = ""
aws_secret_access_key = ""
CharacterMeshUpload_Not_Preprocessed = 'CharacterMeshUpload_Not_Preprocessed'
CharacterMeshUpload_Sucess_Preprocessed = 'CharacterMeshUpload_Sucess_Preprocessed'
ChracterMeshFromWeb_s3 = 'character-mesh-from-web'
CharacterMeshPreprocessed_s3 = "character-mesh-preprocessed"

local_fbx_download_dir = r'C:\Users\user\Downloads'
project_skeletal_mesh_import_dir = r"C:\Users\user\Documents\Unreal Projects\PythonTest\Content\SkeletalMesh"
skeleton_path = "/Game/Mesh/Girl/Jody_Skeleton.Jody_Skeleton"
SkeletalMeshFolderInUE4 = "/Game/SkeletalMesh"

dynamo_table = boto3.resource('dynamodb',
                              region_name=region_name,
                              aws_access_key_id=aws_access_key_id,
                              aws_secret_access_key=aws_secret_access_key,
                              ).Table(CharacterMeshUpload_Not_Preprocessed)
dynamodb_cli = boto3.client('dynamodb',
                            region_name=region_name,
                            aws_access_key_id=aws_access_key_id,
                            aws_secret_access_key=aws_secret_access_key)

s3_client = boto3.client('s3',
                         region_name=region_name,
                         aws_access_key_id=aws_access_key_id,
                         aws_secret_access_key=aws_secret_access_key)


# def import_mesh(skeleton_path, fbx_file, destination_path, destination_name):
#     # local_directory = r"C:\Program Files\Epic Games\UE_4.25\Engine\Content\Python"
#     options = AssetFunctions.buildSkeletalMeshImportOptions(skeleton_path=skeleton_path)
#     task = AssetFunctions.buildImportTask(filename=fbx_file, destination_path=destination_path,
#                                           destination_name=destination_name,
#                                           options=options)
#     imported_asset_path = AssetFunctions.executeImportTasks(tasks=[task])
#     return imported_asset_path


def fetch_from_CharacterMeshUpload_Not_Preprocessed():
    result = []
    try:
        result = dynamo_table.scan()
    except Exception as e:
        print(e)
    if result:
        return result["Items"]
    else:
        return result


def delete_from_CharacterMeshUpload_Not_Preprocessed_and_append_in_CharacterMeshUpload_Sucess_Preprocessed(Items):
    # deleting item from the table
    with dynamo_table.batch_writer() as batch:
        i = 0
        for item in Items:
            now_time = datetime.now().strftime('%y-%m-%d %H:%M:%S')  # tz=timezone.utc
            try:
                batch.delete_item(Key={'filename': item['filename'], 'UploadTime': item["UploadTime"]})
                i += 1
                if i == 5:
                    break
                res = dynamodb_cli.put_item(TableName=CharacterMeshUpload_Sucess_Preprocessed,
                                            Item={
                                                'filename': {'S': item['filename']},
                                                'ID': {'N': item['ID']},
                                                'user_email': {'S': item['user_email']},
                                                'user_firstname': {'S': item['user_firstname']},
                                                'UploadTime': {'S': item["UploadTime"]},
                                                'PreprocessedTime': {'S': str(now_time)},
                                            })
            except Exception as e:
                print(e)


def download_mesh(s3_filename, local_path="0.fbx"):
    try:
        s3_client.download_file(ChracterMeshFromWeb_s3, Key=s3_filename, Filename=local_path)
        # it waits till the donwload completes then moves the execution to forward
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise


def delete_mesh(s3_filename):
    try:
        s3_client.delete_object(Bucket=ChracterMeshFromWeb_s3, Key=s3_filename)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist or previously deleted.")
        else:
            raise


def upload_dir_to_s3(ID):
    destination = str(ID)
    user_mesh_import_dir = os.path.join(project_skeletal_mesh_import_dir, destination)

    # enumerate local files recursively
    for root, dirs, files in os.walk(user_mesh_import_dir):
        for filename in files:
            # construct the full local path
            local_path = os.path.join(root, filename)

            # construct the full Dropbox path
            relative_path = os.path.relpath(local_path, user_mesh_import_dir)
            s3_path = os.path.join(destination, relative_path).replace("\\", "/")

            print('Searching "%s" in "%s"' % (s3_path, CharacterMeshPreprocessed_s3))
            try:
                s3_client.head_object(Bucket=CharacterMeshPreprocessed_s3, Key=s3_path)
                print("Path found on S3! Deleting %s..." % s3_path)
                try:
                    s3_client.delete_object(Bucket=CharacterMeshPreprocessed_s3, Key=s3_path)
                    try:
                        print("Uploading new to s3 %s..." % s3_path)
                        s3_client.upload_file(local_path, Bucket=CharacterMeshPreprocessed_s3, Key=s3_path)
                    except ClientError as e:
                        logging.error(e)
                except:
                    print("Unable to delete from s3 %s..." % s3_path)
            except:
                try:
                    print("Uploading to s3 %s..." % s3_path)
                    s3_client.upload_file(local_path, Bucket=CharacterMeshPreprocessed_s3, Key=s3_path)
                except ClientError as e:
                    logging.error(e)


def download_dir(prefix, local, bucket=CharacterMeshPreprocessed_s3, client=s3_client):
    """
    params:
    - prefix: pattern to match in s3
    - local: local path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket': bucket,
        'Prefix': prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        try:
            print("Downloading {}".format(dest_pathname))
            client.download_file(bucket, k, dest_pathname)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise

                
                

def sendEmail(user_email, display_name):
    ses_client = boto3.client('ses',
                            region_name=region_name,
                            aws_access_key_id=aws_access_key_id,
                            aws_secret_access_key=aws_secret_access_key)
    body = """
    Hi {},
    you have successfully uploaded your character.
    Thanks for making this far. Your character will be preprocessed with our automated system before it will be available to you for exploring any map. We will notify you on processing complete via an email notification. It usually takes 24 hours. Wait till that time and watch our trailers of different maps.

    Thanks
    https://castme.life
    support@castme.life
    +19562157882""".format(display_name)
    try:
        ses_client.send_email(
            Source="support@castme.life",
            Destination={
                'ToAddresses': [user_email],
                # 'CcAddresses': [], // don't put them here if they are empty
                # 'BccAddresses': [],
            },
            ReplyToAddresses=['support@castme.life'],
            Message={
                'Subject': {
                    'Data': 'Character Upload successful, castme.life',
                    'Charset': 'utf-8'
                },
                'Body': {
                    'Text': {'Data': body, 'Charset': 'utf-8'},
                    'Html': {'Data': body, 'Charset': 'utf-8'}
                }
            }
        )
    except Exception as e:
        print(e)


def main():
    Items = fetch_from_CharacterMeshUpload_Not_Preprocessed()
    for item in Items:
        '''Downloading skeletal mesh'''
        fbx_file = os.path.join(local_fbx_download_dir, item['filename'])
        download_mesh(s3_filename=item['filename'], local_dir=fbx_file)

        '''Run import script and wait for completion, run more scripts for fine tuning'''
        destination_path = SkeletalMeshFolderInUE4 + "/" + item["ID"]
#         imported_asset_path = import_mesh(skeleton_path, fbx_file, destination_path, destination_name=item["ID"])

#         if not AssetFunctions.saveDirectory(path=destination_path):
#             print("saveDirectory failing for {}".format(destination_path))
#             return

        '''Uploading completely fine tuned character to s3 as a complete folder'''
        upload_dir_to_s3(ID=item['ID'])

        '''Deleting local .fbx file, imported folder in UE4 and from CharacterMeshPreprocessed_s3 bucket'''
        try:
            if os.path.exists(fbx_file):
                os.remove(fbx_file)
            else:
                print("The file does not exist, or have already got deleted.")
        except Exception as e:
            print(e)

        try:
            # Removes the specified directory, all subdirectories, and all files.
#             if AssetFunctions.directoryExist(path=destination_path):
#                 AssetFunctions.deleteCompleteDir(path=destination_path)
            shutil.rmtree(os.path.join(project_skeletal_mesh_import_dir, item['ID']))
        except Exception as e:
            print(e)

        # delete_mesh(s3_filename=item['filename'])

    # updating both of the dynamodb tables
    delete_from_CharacterMeshUpload_Not_Preprocessed_and_append_in_CharacterMeshUpload_Sucess_Preprocessed(Items=Items)


In [14]:
download_mesh('1.fbx')
print("hello")

hello
